EDA 
Preparación de datos
Realizado el análisis exploratorio de datos de acuerdo con la descripción del dataset y la información entregada , se comparte un paso a paso  el proceso de EDA y preparación de datos, lo cual es crucial antes de desarollar el modelamiento.

In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 

In [2]:
#Exploramos el dataset mostrando los priemor 5 registros con head()
df=pd.read_csv("movie_credits.csv")
df.head()
#Verificando nuestro dataframe se puede ver que en movies_credits.csv tenemos belongs_to_collection ,
# budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date	,
# runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return	y crew

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return,crew
0,Toy Story Collection,30000000.0,Animation,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,...,81.0,English,Released,NaN,Toy Story,7.7,5415.0,1995.0,12.451801,John Lasseter
1,NaN,65000000.0,Adventure,8844,en,When siblings Judy and Peter discover an encha...,17.015539,TriStar Pictures,United States of America,1995-12-15,...,104.0,English,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,1995.0,4.043035,Larry J. Franco
2,Grumpy Old Men Collection,0.0,Romance,15602,en,A family wedding reignites the ancient feud be...,11.712900,Warner Bros.,United States of America,1995-12-22,...,101.0,English,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,1995.0,0.000000,Howard Deutch
3,NaN,16000000.0,Comedy,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,Twentieth Century Fox Film Corporation,United States of America,1995-12-22,...,127.0,English,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,1995.0,5.090760,Forest Whitaker
4,Father of the Bride Collection,0.0,Comedy,11862,en,Just when George Banks has recovered from his ...,8.387519,Sandollar Productions,United States of America,1995-02-10,...,106.0,English,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,1995.0,0.000000,Alan Silvestri


In [3]:
### Informacion de cada una de las columnas
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45451 entries, 0 to 45450
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4497 non-null   object 
 1   budget                 45451 non-null  float64
 2   genres                 45451 non-null  object 
 3   id                     45451 non-null  int64  
 4   original_language      45440 non-null  object 
 5   overview               44510 non-null  object 
 6   popularity             45451 non-null  float64
 7   production_companies   45451 non-null  object 
 8   production_countries   45451 non-null  object 
 9   release_date           45451 non-null  object 
 10  revenue                45451 non-null  float64
 11  runtime                45205 non-null  float64
 12  spoken_languages       45230 non-null  object 
 13  status                 45371 non-null  object 
 14  tagline                20425 non-null  object 
 15  ti

In [4]:
#para tal modelamiento se eliminara algunas columnas que no son necesarias ya que el modelo 
# consiste en recomendar películas a los usuarios basándose en películas similares, 
# por lo que se debe encontrar la similitud de puntuación entre esa película y el resto de películas, se ordenarán según el score de similaridad 
df = df.drop(['belongs_to_collection' ,
'original_language','overview','production_companies','production_countries','release_date',
'spoken_languages','status','tagline','crew'], axis=1 )


In [5]:
#valores nulos de cada columna
df.isnull().sum()

budget            0
genres            0
id                0
popularity        0
revenue           0
runtime         246
title             0
vote_average      0
vote_count        0
release_year      0
return            0
dtype: int64

Estadísticas descriptivas

Mediana : El valor medio en las Columnas, también llamado percentil 50. o 2do cuartil.

1er Cuartil : El percentil 25.

3er Cuartil : El percentil 75.

Mínimo : La observación más pequeña en columnas.

Máximo : La observación más grande en columnas.

In [6]:
#Para ver las variables estadisticas  descriptivas usamos la funcion describe()
df.describe(include='all')

,budget,genres,id,popularity,revenue,runtime,title,vote_average,vote_count,release_year,return
count,4.545100e+04,45451,45451.000000,45451.000000,4.545100e+04,45205.000000,45451,45451.000000,45451.000000,45451.000000,4.545100e+04
unique,NaN,21,NaN,NaN,NaN,NaN,42195,NaN,NaN,NaN,NaN
top,NaN,Drama,NaN,NaN,NaN,NaN,Blackout,NaN,NaN,NaN,NaN
freq,NaN,11989,NaN,NaN,NaN,NaN,13,NaN,NaN,NaN,NaN
mean,4.231294e+06,NaN,107984.599745,2.926887,1.121930e+07,94.181905,NaN,5.624085,109.989703,1991.882027,6.589542e+02
std,1.742942e+07,NaN,112111.336735,6.006458,6.433984e+07,38.329504,NaN,1.915424,491.352336,24.057726,7.463165e+04
min,0.000000e+00,NaN,2.000000,0.000000,0.000000e+00,0.000000,NaN,0.000000,0.000000,1874.000000,0.000000e+00
25%,0.000000e+00,NaN,26367.500000,0.389095,0.000000e+00,85.000000,NaN,5.000000,3.000000,1978.000000,0.000000e+00
50%,0.000000e+00,NaN,59871.000000,1.131386,0.000000e+00,95.000000,NaN,6.000000,10.000000,2001.000000,0.000000e+00
75%,0.000000e+00,NaN,156327.500000,3.694370,0.000000e+00,107.000000,NaN,6.800000,34.000000,2010.000000,0.000000e+00


In [ ]:
sns.boxplot(x='GarageQual',y=‘SalePrice’,data=df)
plt.show()

In [7]:
import sklearn as sk
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
corpus = df['title'].tolist()

In [11]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')
def stemming_tokenizer(str_input):
    words = re.sub(r'[^a-zA-Z]{2,}', str_input).lower().split()
    words = [stemmer.stem(word) for word in words]
    return words

In [12]:
stemming_tokenizer('love')

TypeError: sub() missing 1 required positional argument: 'string'

In [9]:
titles_df=pd.DataFrame(corpus,columns=['title'])
cos_sim_df=pd.concat([titles_df,sim_posts,sim_value],axis=1)
cos_sim_df = cos_sim_df[cos_sim_df['Coseno Sim'] > 0]
cos_sim_df

NameError: name 'sim_posts' is not defined